# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import google_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file="../WeatherPy/cities.csv"
weather_df=weather_df = pd.read_csv(weather_csv_file)

weather_df.head()



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Albany,28,US,1615988895,58,42.6001,-73.9662,34.00,2.48
1,Pevek,100,RU,1615988613,98,69.7008,170.3133,-20.97,4.32
2,Mumford,82,GH,1615988830,72,5.2618,-0.7590,84.47,16.60
3,Guaraniaçu,40,BR,1615988896,73,-25.1008,-52.8781,69.80,9.22
4,Saint George,1,US,1615988718,93,37.1041,-113.5841,37.00,1.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps.
gmaps.configure(api_key=google_api_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
13,Beihai,0,CN,1615988854,83,21.4833,109.1000,73.56,2.26
16,São Filipe,0,CV,1615988616,74,14.8961,-24.4956,72.45,6.80
21,Tura,0,IN,1615988677,22,25.5198,90.2201,71.64,2.04
30,Muros,0,ES,1615988903,38,42.7762,-9.0603,71.01,5.01
40,Niquero,0,CU,1615988905,63,20.0431,-77.5828,77.47,6.08
87,Waddān,0,LY,1615988915,24,29.1614,16.1390,70.27,7.09
183,Nueva Loja,0,EC,1615988939,94,0.0847,-76.8828,73.40,2.30
201,Puerto Padre,0,CU,1615988943,83,21.1950,-76.6028,71.60,2.30
214,Nautla,0,MX,1615988946,80,20.2167,-96.7833,76.86,3.22
255,Teknāf,0,BD,1615988956,59,20.8624,92.3058,76.03,7.43


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": google_api_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 13: Beihai.
Closest hotel is 21 Degree Hostel.
------------
Retrieving Results for Index 16: São Filipe.
Closest hotel is Tortuga B&B.
------------
Retrieving Results for Index 21: Tura.
Closest hotel is Hotel RIKMAN Continental.
------------
Retrieving Results for Index 30: Muros.
Closest hotel is Apartment Av. Castelao.
------------
Retrieving Results for Index 40: Niquero.
Closest hotel is Islazul Niquero.
------------
Retrieving Results for Index 87: Waddān.
Closest hotel is محل الشريف محمد العايب.
------------
Retrieving Results for Index 183: Nueva Loja.
Closest hotel is Limoncocha Great Hotel.
------------
Retrieving Results for Index 201: Puerto Padre.
Closest hotel is El Paseo.
------------
Retrieving Results for Index 214: Nautla.
Closest hotel is Casa Blanca.
------------
Retrieving Results for Index 255: Teknāf.
Closest hotel is Forestry Guesthouse.
------------
Retrieving Results for Index 268: Sabha.
Closest hotel is Foundation Taaj Asala for 

In [21]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
13,Beihai,0,CN,1615988854,83,21.4833,109.1000,73.56,2.26,21 Degree Hostel
16,São Filipe,0,CV,1615988616,74,14.8961,-24.4956,72.45,6.80,Tortuga B&B
21,Tura,0,IN,1615988677,22,25.5198,90.2201,71.64,2.04,Hotel RIKMAN Continental
30,Muros,0,ES,1615988903,38,42.7762,-9.0603,71.01,5.01,Apartment Av. Castelao
40,Niquero,0,CU,1615988905,63,20.0431,-77.5828,77.47,6.08,Islazul Niquero
87,Waddān,0,LY,1615988915,24,29.1614,16.1390,70.27,7.09,محل الشريف محمد العايب
183,Nueva Loja,0,EC,1615988939,94,0.0847,-76.8828,73.40,2.30,Limoncocha Great Hotel
201,Puerto Padre,0,CU,1615988943,83,21.1950,-76.6028,71.60,2.30,El Paseo
214,Nautla,0,MX,1615988946,80,20.2167,-96.7833,76.86,3.22,Casa Blanca
255,Teknāf,0,BD,1615988956,59,20.8624,92.3058,76.03,7.43,Forestry Guesthouse


In [9]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Add marker layer ontop of heat map


# Display figure
